In [ ]:
# Course: Massive Data Fundamentals 502

# Authors: Yuxuan Yao(yy560), Zihao Zhou(zz267), Kuiyu Zhu(kz175), Guiming Xu(gx26)

# Dataset: Global Surface Summary of Day: https://registry.opendata.aws/noaa-gsod/

# This python file aims at predicting rain or not by Logistic Regression and Decision Tree.

# The evaluation method is Binary Classification Evaluator.

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test SparkSession") \
     .getOrCreate()
from pyspark.conf import SparkConf
conf = SparkConf()
print(conf.toDebugString())

spark.app.name=Test SparkSession
spark.blacklist.decommissioning.enabled=true
spark.blacklist.decommissioning.timeout=1h
spark.decommissioning.timeout.threshold=20
spark.default.parallelism=16
spark.driver.extraClassPath=/usr/lib/hadoop-lzo/lib/*:/usr/lib/hadoop/hadoop-aws.jar:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/*:/usr/share/aws/emr/emrfs/auxlib/*:/usr/share/aws/emr/goodies/lib/emr-spark-goodies.jar:/usr/share/aws/emr/security/conf:/usr/share/aws/emr/security/lib/*:/usr/share/aws/hmclient/lib/aws-glue-datacatalog-spark-client.jar:/usr/share/java/Hive-JSON-Serde/hive-openx-serde.jar:/usr/share/aws/sagemaker-spark-sdk/lib/sagemaker-spark-sdk.jar:/usr/share/aws/emr/s3select/lib/emr-s3-select-spark-connector.jar
spark.driver.extraJavaOptions=-XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=70 -XX:MaxHeapFreeRatio=70 -XX:+CMSClassUnloadingEnabled -XX:OnOutOfMemoryError='kill -9 %p'
spark.driver.extraLibraryPath=/usr/lib/hadoop/l

In [13]:
# load data
data = spark.read.option("header", "true").csv("s3://aws-gsod/*/*.csv")

In [14]:
data.cache()

DataFrame[ID: string, USAF: string, WBAN: string, Elevation: string, Country_Code: string, Latitude: string, Longitude: string, Date: string, Year: string, Month: string, Day: string, Mean_Temp: string, Mean_Temp_Count: string, Mean_Dewpoint: string, Mean_Dewpoint_Count: string, Mean_Sea_Level_Pressure: string, Mean_Sea_Level_Pressure_Count: string, Mean_Station_Pressure: string, Mean_Station_Pressure_Count: string, Mean_Visibility: string, Mean_Visibility_Count: string, Mean_Windspeed: string, Mean_Windspeed_Count: string, Max_Windspeed: string, Max_Gust: string, Max_Temp: string, Max_Temp_Quality_Flag: string, Min_Temp: string, Min_Temp_Quality_Flag: string, Precipitation: string, Precip_Flag: string, Snow_Depth: string, Fog: string, Rain_or_Drizzle: string, Snow_or_Ice: string, Hail: string, Thunder: string, Tornado: string]

In [15]:
# count data
data.count()

140841694

In [89]:
# see the clolumns we may need
data.printSchema()

root
 |-- ID: string (nullable = true)
 |-- USAF: string (nullable = true)
 |-- WBAN: string (nullable = true)
 |-- Elevation: string (nullable = true)
 |-- Country_Code: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Mean_Temp: string (nullable = true)
 |-- Mean_Temp_Count: string (nullable = true)
 |-- Mean_Dewpoint: string (nullable = true)
 |-- Mean_Dewpoint_Count: string (nullable = true)
 |-- Mean_Sea_Level_Pressure: string (nullable = true)
 |-- Mean_Sea_Level_Pressure_Count: string (nullable = true)
 |-- Mean_Station_Pressure: string (nullable = true)
 |-- Mean_Station_Pressure_Count: string (nullable = true)
 |-- Mean_Visibility: string (nullable = true)
 |-- Mean_Visibility_Count: string (nullable = true)
 |-- Mean_Windspeed: string (nullable = true)
 |-- Mean_Windspeed_C

In [41]:
# data cleaning and processing
from pyspark.sql import functions
df = data.select(*(functions.col(col).cast("float").alias(col) for col in data.columns))
# data selection for classification
# predict rain or drizzle
cols = ['Elevation','Latitude', 'Longitude', 'Month', 'Mean_Temp', 'Max_Temp','Min_Temp','Mean_Dewpoint', 'Mean_Sea_Level_Pressure', 
        'Mean_Station_Pressure', 'Mean_Visibility','Fog', 'Mean_Windspeed', 'Precipitation','Rain_or_Drizzle']
df2 = df.select(*cols)
df2 = df2.na.drop().cache()
df2.printSchema()

root
 |-- Elevation: float (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- Month: float (nullable = true)
 |-- Mean_Temp: float (nullable = true)
 |-- Max_Temp: float (nullable = true)
 |-- Min_Temp: float (nullable = true)
 |-- Mean_Dewpoint: float (nullable = true)
 |-- Mean_Sea_Level_Pressure: float (nullable = true)
 |-- Mean_Station_Pressure: float (nullable = true)
 |-- Mean_Visibility: float (nullable = true)
 |-- Fog: float (nullable = true)
 |-- Mean_Windspeed: float (nullable = true)
 |-- Precipitation: float (nullable = true)
 |-- Rain_or_Drizzle: float (nullable = true)



In [42]:
# the features without the label
num_cols = df2.columns[:-1]
print(num_cols)
df2 = df2.withColumnRenamed('Rain_or_Drizzle', 'label')

['Elevation', 'Latitude', 'Longitude', 'Month', 'Mean_Temp', 'Max_Temp', 'Min_Temp', 'Mean_Dewpoint', 'Mean_Sea_Level_Pressure', 'Mean_Station_Pressure', 'Mean_Visibility', 'Fog', 'Mean_Windspeed', 'Precipitation']


In [43]:
# logistic regression model
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

# stages for pipeline
print('stages begin')
stages = []
assemblerInputs=num_cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

stages begin


In [44]:
# build pipeline
from pyspark.ml import Pipeline
print('pipeline begin')
cols = df2.columns
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df2)
df2 = pipelineModel.transform(df2)
selectedCols = ['features']+cols
df2 = df2.select(selectedCols)
train, test = df2.randomSplit([0.7, 0.3])
print('pipeline done')

pipeline begin
pipeline done


In [84]:
# LogisticRegression modeling
from pyspark.ml.classification import LogisticRegression
print('Linear Regression Started')
LR = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=15)
LR_model = LR.fit(train)
print('Linear Regression Completed')

Linear Regression Started
Linear Regression Completed


In [85]:
# view the result of train dataset (AUC)
trainingSummary = LR_model.summary
roc = trainingSummary.roc.toPandas()
print('Training set ROC for LR: ' + str(trainingSummary.areaUnderROC))

Training set ROC for LR: 0.6991243570426643


In [86]:
# view the result of test dataset
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions_LR = LR_model.transform(test)
evaluator = BinaryClassificationEvaluator()
print("Testing set Area Under ROC for Logistic Regression: " + str(evaluator.evaluate(predictions_LR, {evaluator.metricName: "areaUnderROC"})))

Testing set Area Under ROC for Logistic Regression: 0.6993817014804423


In [56]:
# DecisionTreeClassifier modeling
# same process as LogisticRegression modeling

from pyspark.ml.classification import DecisionTreeClassifier

# train
print('Decision Tree Started')
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.select('label', 'rawPrediction', 'prediction').show(10)

print('Decision Tree Completed')

Decision Tree Started
+-----+--------------------+----------+
|label|       rawPrediction|prediction|
+-----+--------------------+----------+
|  0.0|[1.7490974E7,1970...|       0.0|
|  0.0|[1.7490974E7,1970...|       0.0|
|  0.0|[1.7490974E7,1970...|       0.0|
|  0.0|[1.7490974E7,1970...|       0.0|
|  0.0|[1.7490974E7,1970...|       0.0|
|  0.0|[1.7490974E7,1970...|       0.0|
|  0.0|[1.7490974E7,1970...|       0.0|
|  0.0|[1.7490974E7,1970...|       0.0|
|  0.0|[1.7490974E7,1970...|       0.0|
|  0.0|[1.7490974E7,1970...|       0.0|
+-----+--------------------+----------+
only showing top 10 rows

Decision Tree Completed


In [57]:
evaluator = BinaryClassificationEvaluator()

print("Testing set Area Under ROC for Decision Tree: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Testing set Area Under ROC for Decision Tree: 0.7994905015933091
